# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
state=12345
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [2]:
data_0 = pd.read_csv('geo_data_0.csv')
data_1 = pd.read_csv('geo_data_1.csv')
data_2 = pd.read_csv('geo_data_2.csv')
display(data_0.sample(10))
display(data_1.sample(10))
display(data_2.sample(10))

,id,f0,f1,f2,product
40694,Z3QyJ,-0.897825,0.019472,-0.647829,66.322110
17324,bpBH0,0.460428,0.600222,2.172946,98.072733
88410,oQ6pL,0.263389,-0.016925,5.925583,79.976867
58804,BMlfw,0.965840,0.451677,0.787505,46.995587
6363,eGskD,-0.717827,0.607322,2.437840,121.776004
65997,oXTYS,1.960127,0.190192,1.862871,134.838776
25282,aW7fo,-0.841649,0.135716,-0.806809,34.957048
59340,z3lPn,1.213096,-0.355636,-0.848810,27.956479
98506,BTkdN,1.228604,-0.633343,5.605382,153.355831
41522,ut2xN,0.644139,0.842346,0.700427,52.991770


,id,f0,f1,f2,product
49321,aGLMU,11.765493,-3.051704,1.005144,26.953261
28625,T1h65,10.119817,-6.016601,0.995910,26.953261
80592,HV00D,9.306168,-6.494699,2.995691,80.859783
82017,4VDxR,5.314630,-7.265467,3.994412,107.813044
13950,ui25y,-5.643956,-9.964143,3.996956,110.992147
94266,tVrry,1.021770,-5.832455,0.995981,30.132364
35194,QZ426,12.683192,-2.237029,1.992664,53.906522
15071,yJngD,12.018297,-14.376039,1.005714,26.953261
72557,YCjol,8.218453,-3.104388,1.991578,53.906522
44036,xpKWf,0.879461,-5.171192,5.009278,134.766305


,id,f0,f1,f2,product
16634,Ypb45,2.034009,-2.865780,1.623667,134.252360
20449,iwbV3,-3.354662,0.052315,0.149168,107.311857
72925,QYn72,1.052491,2.530617,-6.656610,73.374758
92426,eEUTj,-1.066212,0.308193,3.189239,31.124822
27410,DYP3P,1.220402,-0.944619,5.040419,123.440113
67250,FLl27,2.784895,-2.797032,-2.378223,94.821409
69002,HVqSi,4.348273,0.204292,1.401250,42.503676
97700,0hNjp,-1.683178,-0.795255,1.363773,5.763997
91980,3NUr7,-1.184275,-0.369046,-0.284876,0.041786
52370,PKA42,-1.779276,2.717118,5.842610,155.613802


In [3]:
data_0.info()
data_1.info()
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [4]:
print(data_0.duplicated().sum())
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())

0
0
0


В данных отсутствуют дубликаты

In [5]:
data_0.isna().sum()
data_1.isna().sum()
data_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

В данных отсутствуют пропуски

## Обучение и проверка модели

 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

In [6]:

# Для региона 1
features_0 = data_0[['f0', 'f1', 'f2']]  # Признаки
target_0 = data_0['product']  # Целевая переменная

X_train_0, X_valid_0, y_train_0, y_valid_0 = train_test_split(
    features_0, target_0, 
    test_size=0.25,  # 25% на валидацию
    random_state=state # Для воспроизводимости
)
# Аналогично для региона 2
features_1 = data_1[['f0', 'f1', 'f2']]  # Признаки
target_1 = data_1['product']  # Целевая переменная
X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(
    features_1, target_1, 
    test_size=0.25, 
    random_state=state 
)
# Аналогично для региона 3

features_2 = data_2[['f0', 'f1', 'f2']]  # Признаки
target_2 = data_2['product']  # Целевая переменная
X_train_2, X_valid_2, y_train_2, y_valid_2 = train_test_split(
    features_2, target_2, 
    test_size=0.25, 
    random_state=state 
)

2.2. Обучите модель и сделайте предсказания на валидационной выборке.

In [7]:
# Создаем модель для каждого региона
model_0 = LinearRegression()
model_1 = LinearRegression()
model_2 = LinearRegression()

# Обучаем модели
model_0.fit(X_train_0, y_train_0)
model_1.fit(X_train_1, y_train_1)
model_2.fit(X_train_2, y_train_2)

# Делаем предсказания на валидационной выборке
predictions_0 = model_0.predict(X_valid_0)
predictions_1 = model_1.predict(X_valid_1)
predictions_2 = model_2.predict(X_valid_2)

 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.

In [8]:
results_0 = pd.DataFrame({'id':data_0.iloc[X_valid_0.index]['id'].values,
                         'true':y_valid_0.values,
                         'predictions':predictions_0,
                         'residual':y_valid_0.values - predictions_0})
results_1 = pd.DataFrame({'id':data_1.iloc[X_valid_1.index]['id'].values,
                         'true':y_valid_1.values,
                         'predictions':predictions_1,
                         'residual':y_valid_1.values - predictions_1})
results_2 = pd.DataFrame({'id':data_2.iloc[X_valid_2.index]['id'].values,
                         'true':y_valid_2.values,
                         'predictions':predictions_2,
                         'residual':y_valid_2.values - predictions_2})
display(results_0)
display(results_1)
display(results_2)

,id,true,predictions,residual
0,7Z2gx,10.038645,95.894952,-85.856307
1,b2HU8,114.551489,77.572583,36.978906
2,Y0dXD,132.603635,77.892640,54.710996
3,Mje59,169.072125,90.175134,78.896991
4,yqXxU,122.325180,70.510088,51.815092
...,...,...,...,...
24995,CzptN,170.116726,103.037104,67.079623
24996,cT9bJ,93.632175,85.403255,8.228920
24997,Y7xR6,127.352259,61.509833,65.842426
24998,4WJRf,99.782700,118.180397,-18.397697


,id,true,predictions,residual
0,aTbUt,80.859783,82.663314,-1.803531
1,CK34V,53.906522,54.431786,-0.525264
2,ioXCO,30.132364,29.748760,0.383604
3,xk6tF,53.906522,53.552133,0.354389
4,F9AmG,0.000000,1.243856,-1.243856
...,...,...,...,...
24995,fJBfd,137.945408,136.869211,1.076197
24996,paaof,110.992147,110.693465,0.298682
24997,tOADJ,137.945408,137.879341,0.066067
24998,AiZ1G,84.038886,83.761966,0.276920


,id,true,predictions,residual
0,DMlgl,61.212375,93.599633,-32.387258
1,tbrRC,41.850118,75.105159,-33.255040
2,JRs1w,57.776581,90.066809,-32.290228
3,bidOx,100.053761,105.162375,-5.108614
4,AcBBB,109.897122,115.303310,-5.406189
...,...,...,...,...
24995,y2n7n,28.492402,78.765887,-50.273485
24996,aBi2q,21.431303,95.603394,-74.172091
24997,CMYf3,125.487229,99.407281,26.079948
24998,HzkKW,99.422903,77.779912,21.642990


 2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

In [9]:
# Функция для расчета RMSE
def metrics(reg, true, predicted):
    rmse = mean_squared_error(true, predicted, squared=False)
    avg = predicted.mean()

    
    print(f"\n{reg}:")
    print(f"  Средний предсказанный запас: {avg:.2f} тыс. баррелей")
    print(f"  RMSE: {rmse:.2f}")
    
    return rmse, avg
# Рассчитываем для каждого региона
rmse_0, avg_pred_0 = metrics("Регион 1", y_valid_0, predictions_0)
rmse_1, avg_pred_1 = metrics("Регион 2", y_valid_1, predictions_1)
rmse_2, avg_pred_2 = metrics("Регион 3", y_valid_2, predictions_2)


Регион 1:
  Средний предсказанный запас: 92.59 тыс. баррелей
  RMSE: 37.58

Регион 2:
  Средний предсказанный запас: 68.73 тыс. баррелей
  RMSE: 0.89

Регион 3:
  Средний предсказанный запас: 94.97 тыс. баррелей
  RMSE: 40.03


2.5. Проанализируйте результаты.

Лучше всего себя показала модель, обученная на данных второго региона

## Подготовка к расчёту прибыли

 3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.

In [10]:
BUDGET = 10_000_000_000  # Бюджет: 10 млрд рублей
WELLS_TO_DRILL = 200  # Количество скважин для разработки (отбираем лучшие)
TOTAL_POINTS = 500  # Всего исследуемых точек
REVENUE_PER_BARREL = 450000  # Доход с одного барреля, рублей

 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе. 

In [11]:
BREAK_EVEN_VOLUME = BUDGET / REVENUE_PER_BARREL/WELLS_TO_DRILL
print(f"\nМинимальный объем для безубыточности одной скважины: {BREAK_EVEN_VOLUME:.2f} тыс. баррелей")


Минимальный объем для безубыточности одной скважины: 111.11 тыс. баррелей


 3.3. Напишите выводы по этапу подготовки расчёта прибыли.

In [12]:
avg_volume_all_0 = data_0['product'].mean()
avg_volume_all_1 = data_1['product'].mean()
avg_volume_all_2 = data_2['product'].mean()
print(avg_volume_all_0)
print(avg_volume_all_1)
print(avg_volume_all_2)

92.50000000000001
68.82500000000002
95.00000000000004


In [13]:
def percent(avg_pred):
    percent = ((BREAK_EVEN_VOLUME - avg_pred) / avg_pred) * 100
    print(percent)
    
percent(avg_pred_0)
percent(avg_pred_1)
percent(avg_pred_2)

20.000032151451684
61.666608898541085
17.002113755145313


**Вывод:**

Сердний предсказанный и реальный объем меньше желаемого минимального объема. Предсказанный средний объм в первом регионе меньше минимального желаемого на 20%. во втором регионе на 61%, а в третьем 17%

## Расчёт прибыли и рисков 

 4.1. Выберите скважины с максимальными значениями предсказаний. 

 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.

 4.3. Рассчитайте прибыль для полученного объёма сырья.

In [15]:
def bootstrap_profit_simple(results_df, random_state=12345):
    state = np.random.RandomState(random_state)
    
    profits = []
    
    for i in range(1000):
        # 1. Случайный выбор 500 скважин с заменой из всех данных
        bootstrap_sample = results_df.sample(n=500, replace=True, random_state=state)
        
        # 2. Сортируем по предсказанным значениям и берем 200 лучших
        sorted_sample = bootstrap_sample.sort_values(by='predictions', ascending=False)
        top_200_true = sorted_sample.head(200)['true']
        
        # 3. Считаем прибыль
        total_volume = top_200_true.sum()
        revenue = total_volume * REVENUE_PER_BARREL
        profit_value = revenue - BUDGET
        
        profits.append(profit_value)
    
    return np.array(profits)

In [16]:
profit_0 = bootstrap_profit_simple(results_0)
profit_1 = bootstrap_profit_simple(results_1)
profit_2 = bootstrap_profit_simple(results_2)

In [17]:

for i, (profits, region_name) in enumerate([(profit_0, 'Регион 1'), 
                                            (profit_1, 'Регион 2'), 
                                            (profit_2, 'Регион 3')]):
    # Средняя прибыль
    profits = np.array(profits)
    mean_profit = profits.mean()
    
    # 95% доверительный интервал
    ci_lower = np.percentile(profits, 2.5)
    ci_upper = np.percentile(profits, 97.5)
    
    # Риск убытков
    risk = (profits < 0).mean() * 100
    
    print(f"\n{region_name}:")
    print(f"  Средняя прибыль: {mean_profit/1e9:.2f} млрд руб.")
    print(f"  95% ДИ: [{ci_lower/1e9:.2f}, {ci_upper/1e9:.2f}] млрд руб.")
    print(f"  Риск убытков: {risk:.2f}%")


Регион 1:
  Средняя прибыль: 0.40 млрд руб.
  95% ДИ: [-0.11, 0.91] млрд руб.
  Риск убытков: 6.90%

Регион 2:
  Средняя прибыль: 0.46 млрд руб.
  95% ДИ: [0.03, 0.85] млрд руб.
  Риск убытков: 1.50%

Регион 3:
  Средняя прибыль: 0.40 млрд руб.
  95% ДИ: [-0.16, 0.95] млрд руб.
  Риск убытков: 7.60%


**Выод** 
На основе анализа рисков и прибыльности, только Регион 2 соответствует установленному критерию допустимости риска (вероятность убытков 1.50% < 2.5%), демонстрируя при этом наиболее высокую среднюю прибыль в 0.46 млрд руб. с 95% доверительным интервалом от 0.03 до 0.85 млрд руб., что делает его единственным подходящим для разработки при заданном ограничении по риску.